In [ ]:
# Question to answer: Is there a corrlation between wide receivers and their offensive systems to thier ppr points
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

from scipy.stats import pearsonr

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

import matplotlib.pyplot as plt

data = pd.read_csv("/kaggle/input/23-24-fantasy-football-dataset/Fantasy Football Dataset.csv")

df = pd.DataFrame(data)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# Keep the columns that will be relevant
df.drop(['DKPt', 'FDPt', 'VBD', '-9999', 'OvRank', 'FantPt','Cmp', '2PM', '2PP', 'Rk', 'Att', 'TD', 'TD.1', 'Yds','Yds.1','Att.1', 'Int', 'Y/A', 'Fmb', 'FL'], axis=1, inplace=True)

df.rename({
    'TD.2' : 'ReceivingTD',
    'TD.3': 'TotalTD',
    'Yds.2' : 'ReceivingYDs',
    'Y/R' : 'YardsPerRecep'
}, axis=1, inplace=True)



In [ ]:
# Drop non wr players since we are only looking at wr
wr_df = df[df['FantPos'] == 'WR' ]

wr_df = wr_df[wr_df['Tm'] !='2TM']

wr_df.head()

In [ ]:
# Add each team's offensive system since our dataset does not include it
team_offense = {
    'BAL': 'RPO',                          # Baltimore Ravens
    'BUF': 'Spread',                       # Buffalo Bills
    'CIN': 'Spread',                       # Cincinnati Bengals
    'CLE': 'Wide Zone Run',                # Cleveland Browns
    'DEN': 'West Coast',                   # Denver Broncos
    'HOU': 'Spread',                       # Houston Texans
    'IND': 'Pro Style',                    # Indianapolis Colts
    'JAX': 'West Coast',                   # Jacksonville Jaguars
    'KAN': 'Spread',                        # Kansas City Chiefs
    'LVR': 'West Coast',                    # Las Vegas Raiders
    'LAC': 'Spread',                       # Los Angeles Chargers
    'MIA': 'Spread',                       # Miami Dolphins
    'NWE': 'Erhardt-Perkins',               # New England Patriots
    'NYJ': 'West Coast',                   # New York Jets
    'PIT': 'Pro Style',                    # Pittsburgh Steelers
    'TEN': 'Zone Run',                     # Tennessee Titans
    'ARI': 'Air Coryell',                  # Arizona Cardinals
    'ATL': 'West Coast',                   # Atlanta Falcons
    'CAR': 'West Coast',                   # Carolina Panthers
    'CHI': 'West Coast',                   # Chicago Bears
    'DAL': 'Pro Style',                    # Dallas Cowboys
    'DET': 'West Coast',                   # Detroit Lions
    'GNB': 'West Coast',                    # Green Bay Packers
    'LAR': 'Zone Run',                     # Los Angeles Rams
    'MIN': 'Zone Run',                     # Minnesota Vikings
    'NOR': 'West Coast',                    # New Orleans Saints
    'NYG': 'Pro Style',                    # New York Giants
    'PHI': 'West Coast',                   # Philadelphia Eagles
    'SFO': 'Zone Run',                      # San Francisco 49ers
    'SEA': 'Zone Run',                     # Seattle Seahawks
    'TAM': 'Air Coryell',                   # Tampa Bay Buccaneers
    'WAS': 'West Coast',                   # Washington Commanders
}

wr_df['OffSys'] = wr_df['Tm'].map(team_offense)

wr_df.head()

In [ ]:
grouped_wr_ppr = wr_df.groupby('OffSys')
summary_stats = pd.DataFrame({
    'Mean PPR': grouped_wr_ppr['PPR'].mean(),
    'Median PPR': grouped_wr_ppr['PPR'].median(),
    'Standard Deviation': grouped_wr_ppr['PPR'].std(),
    '25th Percentile': grouped_wr_ppr['PPR'].quantile(0.25),
    '75th Percentile': grouped_wr_ppr['PPR'].quantile(0.75),
    'Sum PPR' : grouped_wr_ppr['PPR'].sum()
})

print(summary_stats)

Pro style, Air Coryell, Spread, and Zone Run average the most ppr points for WR

In [ ]:
# Create a bar chart
summary_stats[['Mean PPR']].plot(kind='bar', color='skyblue')

# Adding titles and labels
plt.title('Average PPR Points by Offensive System')
plt.xlabel('Offensive System')
plt.ylabel('Average PPR Points')
plt.xticks(rotation=45)  # Rotate labels to improve readability
plt.tight_layout()  # Adjust layout to make room for label rotation

# Show the plot
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.bar(summary_stats.index, summary_stats['Mean PPR'], yerr=summary_stats['Standard Deviation'], capsize=5)
plt.title('Mean PPR by Offensive System with Standard Deviation')
plt.xlabel('Offensive System')
plt.ylabel('Mean PPR')
plt.xticks(rotation=45)
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
grouped_wr_tgt = wr_df.groupby('OffSys')
summary_stats_tgt = pd.DataFrame({
    'Mean Targets': grouped_wr_tgt['Tgt'].mean(),
    'Median Targets': grouped_wr_tgt['Tgt'].median(),
    'Standard Deviation': grouped_wr_tgt['Tgt'].std(),
    '25th Percentile': grouped_wr_tgt['Tgt'].quantile(0.25),
    '75th Percentile': grouped_wr_tgt['Tgt'].quantile(0.75),
    'Sum Targets' : grouped_wr_tgt['Tgt'].sum()
})

print(summary_stats_tgt)

In [ ]:
summary_stats_tgt[['Mean Targets']].plot(kind='bar', color='blue')

# Adding titles and labels
plt.title('Average Targets by Offensive System')
plt.xlabel('Offensive System')
plt.ylabel('Average Targets')
plt.xticks(rotation=45)  # Rotate labels to improve readability
plt.tight_layout()  # Adjust layout to make room for label rotation

# Show the plot
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.bar(summary_stats_tgt.index, summary_stats_tgt['Mean Targets'], yerr=summary_stats_tgt['Standard Deviation'], capsize=5)
plt.title('Mean Targets by Offensive System with Standard Deviation')
plt.xlabel('Offensive System')
plt.ylabel('Mean Targets')
plt.xticks(rotation=45)
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
# Plot a scatter plot to check linearity 

# Creating the scatter plot

# Merge the two summary DataFrames on offensive system
merged_df = summary_stats_tgt.join(summary_stats, on='OffSys', how='inner', lsuffix='_tgt', rsuffix='_ppr')

# Adding trend line
z = np.polyfit(merged_df['Mean Targets'], merged_df['Mean PPR'], 1)
p = np.poly1d(z)

# Calculate the R^2 value
r_squared = r2_score(merged_df['Mean PPR'], p(merged_df['Mean Targets']))

# Creating the scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(merged_df['Mean Targets'], merged_df['Mean PPR'], alpha=0.7)

# Adding labels for each point
for offsys in merged_df.index:
    plt.text(merged_df.loc[offsys, 'Mean Targets']+0.2, merged_df.loc[offsys, 'Mean PPR'], offsys, 
             fontsize=9, ha='right', va='bottom')

# Adding a trend line
plt.plot(merged_df['Mean Targets'], p(merged_df['Mean Targets']), "r--")

# Annotating the plot with the R^2 value
plt.text(max(merged_df['Mean Targets']), min(merged_df['Mean PPR']), f'$R^2 = {r_squared:.2f}$', 
         fontsize=12, horizontalalignment='right')

# Show plot
plt.grid(True)
plt.show()

Data seems to be linear. PPR points go up with more usage(targets). Proven by r2 score 0f 0.77

In [ ]:
# Calculate pearson correlation and p value

correlation_coef, p_value = pearsonr(merged_df['Mean Targets'], merged_df['Mean PPR'])

print(f'Pearson Correlation Coefficient: {correlation_coef:.2f}')
print(f'p-value: {p_value:.4f}')

A high Correlation Coefficient is a strong indicator of a very strong postive linear realtionship. The p-value of 0.0043 is well below the common alpha level of 0.05, which indicates that the probability of observing such a strong correlation by chance is very low if there were actually no correlation. 

In [ ]:
wr_df = wr_df[wr_df['PPR'] > 0]
wr_df = wr_df[wr_df['YardsPerRecep'] > 0]

In [ ]:
X = summary_stats_tgt[['Mean Targets']]  # Predictor
y = summary_stats['Mean PPR']        # Response

# Initialize and fit the linear regression model
model = LinearRegression()
model.fit(X, y)

# Predict the PPR values
y_pred = model.predict(X)

# Evaluate the model
mse = mean_squared_error(y, y_pred)
r2 = r2_score(y, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')

# Display coefficients
print("Coefficient for Targets:", model.coef_[0])
print("Intercept:", model.intercept_)

While the mse score indicates the model has room for improvement, it's not excessively high given the spread and scale of PPR scores. The error magnitude appears reasonable given the data complexity and the fact that only one predictor (average targets) was used. With an R² value of 0.7678, the model explains about 76.78% of the variance in PPR from the targets across systems. This confirms that while the model is quite good